In [ ]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import datetime

import tensorflow
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score, roc_curve

from imblearn.under_sampling import RandomUnderSampler

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
path_benih = '/content/drive/My Drive/Penelitian Andyka/Kualitass/dataset/'

In [ ]:
kelas = ['broken', 'discolored', 'pure', 'silkcut']
kelas

['broken', 'discolored', 'pure', 'silkcut']

In [ ]:
def get_data(data_dir):
    size = (224,224)
    label = 0
    x = []
    y = []
    for folder in kelas:
        print("Loading {}".format(data_dir+folder))
        for pic in tqdm(glob.glob(data_dir + folder + '/*')):
            im = cv2.imread(pic)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            im = cv2.resize(im, (size))
            im = np.array(im)
            x.append(im)
            y.append(label)
        label+=1
    x = np.array(x, dtype = 'float32')
    y = np.array(y, dtype = 'int32')
    return x,y

In [ ]:
x_test, y_test = get_data(path_benih + 'test/')

Loading /content/drive/My Drive/Penelitian Andyka/Kualitass/dataset/test/broken


100%|██████████| 250/250 [00:03<00:00, 79.39it/s] 


Loading /content/drive/My Drive/Penelitian Andyka/Kualitass/dataset/test/discolored


100%|██████████| 250/250 [00:02<00:00, 88.47it/s] 


Loading /content/drive/My Drive/Penelitian Andyka/Kualitass/dataset/test/pure


100%|██████████| 250/250 [00:02<00:00, 83.68it/s] 


Loading /content/drive/My Drive/Penelitian Andyka/Kualitass/dataset/test/silkcut


100%|██████████| 250/250 [00:03<00:00, 83.22it/s] 


In [ ]:
x_test/=255.

In [ ]:
len(x_test)

1000

In [ ]:
save_dir = '/content/drive/My Drive/Penelitian Andyka/Kualitass/save_models/percobaan/'

In [ ]:
model = load_model(save_dir+"model_5")
y_pred = model.predict(x_test)

32/32 [==============================] - 724s 23s/step


In [ ]:
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
acc_test = accuracy_score(y_test, y_pred)
prec_test = precision_score(y_test, y_pred, average='micro')
rec_test = recall_score(y_test, y_pred, average='micro')
f1score_test = f1_score(y_test, y_pred, average='micro')

In [ ]:
print('Acc Test : ', acc_test)
print('Prec Test : ', prec_test)
print('Rec Test : ', rec_test)
print('F1-Score Test : ', f1score_test)

Acc Test :  0.817
Prec Test :  0.817
Rec Test :  0.817
F1-Score Test :  0.817


In [ ]:
print(classification_report(y_test, y_pred, target_names = ['Broken (Class 0)','Discolored (Class 1)', 'Pure (Class 2)', 'Silkcut (Class 3)']))

                      precision    recall  f1-score   support

    Broken (Class 0)       0.88      0.81      0.84       250
Discolored (Class 1)       0.82      0.79      0.81       250
      Pure (Class 2)       0.93      0.82      0.87       250
   Silkcut (Class 3)       0.69      0.86      0.76       250

            accuracy                           0.82      1000
           macro avg       0.83      0.82      0.82      1000
        weighted avg       0.83      0.82      0.82      1000



In [ ]:
def plot_image(i, predictions_label, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  # defining the plot value array
  if predictions_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f} ({})".format(kelas[predictions_label],
                                np.max(predictions_label),
                                kelas[true_label]),
                                color=color,
                                fontsize=10)

##### SALAH (BENAR), parameter di dalam kurung merupakan kelas sebenarnya sedangakan di luar kurung merupakan prediksi model

In [ ]:
num_rows = 200
num_cols = 5
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, y_pred[i], y_test, x_test)
plt.show()